In [1]:
import wikipediaapi
import pandas as pd
import numpy as np
from multiprocessing import Pool
from datetime import datetime
import os

In [3]:
# Create wikipedia API instance
wiki = wikipediaapi.Wikipedia('hi')

In [4]:
# get English title of given Hindi page
def get_eng_title(page):
    langlinks = page.langlinks
    
    if 'en' in langlinks.keys():
        return langlinks['en'].title
    else:
        return None

In [5]:
def print_links(page,body):
    links = []
    for link in page.links:
        if body.find(link) != -1:
            links.append(link)

    return links

In [6]:
def word_list(hindi_list):
    # list to return hindi-english mappings in
    word_mappings = []
    
    for hindi_title in hindi_list:
        page = wiki.page(hindi_title)
        
        # if page doesn't exist for this hindi title, skip it
        try:
            page.exists()
        except KeyError:
            continue
        
        # get english title of hindi topic
        eng_title = get_eng_title(page)
        # if there is no english title, skip it
        if eng_title is None:
            continue
        # append final map list with english and hindi titles
        word_mappings.append([hindi_title, eng_title])
    return word_mappings

In [7]:
word_list(['ब्रिटेन','ब्रिटेन'])

[['ब्रिटेन', 'Britain'], ['ब्रिटेन', 'Britain']]

In [8]:
# create code mixed corpus by replacing hindi word in body with english one
def find_and_replace(body, word_list):
    for word in word_list:
        body = body.replace(word[0], word[1])
    return body

In [9]:
# open file with english names
names_file = open('cand.txt', "r", encoding = "ISO-8859-1")
names = []

# clean data and append in names list
for name in names_file:
    name = name.replace('\n','').replace(" ","_")
    names.append(name)

In [10]:
# create dataframe of english names
names_df = pd.DataFrame()
names_df['english'] = names

names_df = names_df[0:20]
names_df

,english
0,Naresh_Chandra
1,Ram_Narayan
2,Man_Mohan_Sharma
3,Kalu_Lal_Shrimali
4,Nagendra_Singh
5,Daulat_Singh_Kothari
6,Mohan_Sinha_Mehta
7,Manikya_Lal_Varma
8,Ghanshyam_Das_Birla
9,Padma_Bhushan


In [11]:
def get_hindi_name(x):
    eng_wiki = wikipediaapi.Wikipedia('en')
    page = eng_wiki.page(x)
    
    if 'hi' in page.langlinks:  
        hindi_name = page.langlinks['hi'].title
        print(hindi_name)
        return hindi_name
    return None

In [12]:
names_df['hindi'] = names_df.apply(get_hindi_name, axis = 1)

नरेश चन्द्रा
राम नारायण
मनमोहन शर्मा
के॰ एल॰ श्रीमाली
नगेन्द्र सिंह
दौलत सिंह कोठारी
मोहन सिंह मेहता
माणिक्यलाल वर्मा
घनश्यामदास बिड़ला
पद्म भूषण
कोमल कोठारी
दुर्गा लाल
जगत मेहता
रामनारायण अग्रवाल
लक्ष्मीमल्ल सिंघवी
नटवर सिंह
भोगीलाल पांड्या


In [13]:
names_df

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
5,Daulat_Singh_Kothari,दौलत सिंह कोठारी
6,Mohan_Sinha_Mehta,मोहन सिंह मेहता
7,Manikya_Lal_Varma,माणिक्यलाल वर्मा
8,Ghanshyam_Das_Birla,घनश्यामदास बिड़ला
9,Padma_Bhushan,पद्म भूषण


In [14]:
# names_df = names_df[names_df.hindi != None]
names_df = names_df.replace(to_replace='None', value=np.nan).dropna()

In [15]:
names_df.index = range(len(names_df))
names_df

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
5,Daulat_Singh_Kothari,दौलत सिंह कोठारी
6,Mohan_Sinha_Mehta,मोहन सिंह मेहता
7,Manikya_Lal_Varma,माणिक्यलाल वर्मा
8,Ghanshyam_Das_Birla,घनश्यामदास बिड़ला
9,Padma_Bhushan,पद्म भूषण


In [16]:
names_df.to_csv('parallel_corpus.csv')

In [17]:
word_corpus_list=[]

# given a hindi title, get its mixed corpus
def get_mixed_corpus(hindi_title):
    wiki_wiki = wikipediaapi.Wikipedia('hi')
    page_py = wiki_wiki.page(hindi_title)
    body = page_py.text.replace('\n','').replace('==', '').replace('\u200d', '').replace('।', '')
    print(hindi_title)
    link_list=print_links(page_py,body)
    
    hindi_english_list=word_list(link_list)
    word_corpus_list.append(hindi_english_list)
    replaced_text=find_and_replace(body,hindi_english_list)

    return replaced_text

In [18]:
names_parallel = pd.read_csv('names_parallel.csv')
# names_parallel.drop(names_parallel.columns[names_parallel.columns.str.contains('Unnamed', case=False)], axis=1, inplace=True)

In [19]:
names_parallel

,english,hindi
0,Naresh_Chandra,नरेश चन्द्रा
1,Ram_Narayan,राम नारायण
2,Man_Mohan_Sharma,मनमोहन शर्मा
3,Kalu_Lal_Shrimali,के॰ एल॰ श्रीमाली
4,Nagendra_Singh,नगेन्द्र सिंह
...,...,...
2831,Vinoo_Mankad,वीनू हिम्मतलाल माँकड़
2832,Zaheer_Khan,ज़हीर ख़ान
2833,Rohit_Sharma,रोहित शर्मा
2834,Anjali_Ved_Pathak_Bhagwat,अंजली भगवत


In [20]:
def get_nth_batch(n):
    return names_parallel['hindi'][4*n:(n+1)*4]

def work_on_batch(n):
    # skip nth batch if it is already saved in mixed_corpus folder
    try:
        f = open(f'mixed_corpus/batch_{n}.csv')
        f.close()
        return
    except IOError:
        print(f'Working on batch {n}')
    
    # n = nth batch from 2836 rows
    hindi_names_list = get_nth_batch(n)
    local_mixed_corpus_list = []
    
    for name in hindi_names_list:
        local_mixed_corpus_list.append(get_mixed_corpus(name))
    
    df = pd.DataFrame(local_mixed_corpus_list, columns=['mixed_corpus'])
    filename = f'mixed_corpus/batch_{n}.csv'
    df.to_csv(filename)
    
    print(f'Saved batch {n} file')

def task_finished(task):
    print(task.get_name(), ' finished')
    print(datetime.now().strftime('%H:%M:%S'))

In [21]:
start = datetime.now()
cpu_cores = len(os.sched_getaffinity(0))
n = 709 # number of batches

# create no of processes equal to no of cpu cores
with Pool(cpu_cores) as p:
    p.map(work_on_batch, range(n))
end = datetime.now()

print(end-start)

Working on batch 115
Working on batch 105
Working on batch 69
Working on batch 161
Working on batch 13
Working on batch 68
Working on batch 146
Working on batch 23
ममता शर्मा
फारूक अब्दुल्ला
मोहन लाल सुखाड़िया
विवेक ओबेरॉय
शिल्पा शेट्टी
उमर अब्दुल्ला
के.वी. कामथ
शबाना आज़मी
Working on batch 184
Working on batch 207
करणी सिंह
सदानन्द विश्वनाथ
शारदा सिन्हा
आईसीआईसीआई बैंक
शमा जैन
वरुण चक्रवर्ती
स्नेहा उल्लाल
अशोक कुमार (अभिनेता)
सुमन रंगनाथन
सुनील शेट्टी
अश्विनी नचप्पा
हर्षा भोगले
शेख़ अब्दुल्ला
किरण मजूमदार-शॉ
देवेन्द्र झाझड़िया
फरहा
तबु
महबूबा मुफ़्ती
अंबाती रायडू
एन आर नारायणमूर्ति
Saved batch 69 file
Working on batch 70
शिवलाल यादव
तपन सिन्हा
मुफ़्ती मोहम्मद सईद
जे.जे शोभा
Saved batch 146 file
Working on batch 147
पार्थसारथी शर्मा
वैशाली देसाई
Saved batch 115 file
Working on batch 116
Saved batch 184 file
Working on batch 185
गिरीश नागराजेगौड़ा
अब्बास अली बेग
Saved batch 161 file
Working on batch 162
Saved batch 23 file
Working on batch 24
नन्दन नीलेकणि
गंगोत्री भण्डारी
वसुंधरा दास
अ

Saved batch 123 file
Working on batch 124
तेलंगा खड़िया
ईला अरुण
ठाकुर विश्वनाथ शाहदेव
मास्ती वेंकटेश अयंगार
जगजीत सिंह
निवेदिता मेनन
टिकैत उमराँव सिंह
प्रतिमा बरुआ पांडे
एच सी वर्मा
सीता
Saved batch 253 file
Working on batch 254
ओरेकल कॉर्पोरेशन
तिलका माँझी
बी पी सिन्हा
चंद्रकुमार अग्रवाल
एल एम शर्मा
Saved batch 212 file
Working on batch 213
सैयद शाहनवाज हुसैन
Saved batch 230 file
Working on batch 231
Saved batch 124 file
Working on batch 125
पुदुच्चेरी विधान सभा
अंजना ओम कश्यप
माया कामथ
Saved batch 79 file
Working on batch 80
एन॰ रंगास्वामी
विप्रो
हेमचन्द्र बरुआ
पी. लंकेश
वी॰ नारायणसामी
भारतीय जनता पार्टी
Saved batch 171 file
Working on batch 172
लक्ष्मीनाथ बेजबरुवा
के. पी. पूर्णचंद्र तेजस्वी
कुरुक्षेत्र युद्ध
वीरेन्द्र कुमार भट्टाचार्य
पु. ति. नरसिम्हाचार
दयामणि बारला
अलबर्ट एक्का
आर के नारायण
रणधीर प्रसाद वर्मा
चन्द्रगुप्त मौर्य
मेहदी हसन
Saved batch 30 file
Working on batch 31
संदीप आचार्य
सी॰पी॰ कृष्णन नायर
नलिनीधर भट्टाचार्य
राजा राव
Saved batch 125 file
Working on batch 126
कोच

सी॰ के॰ नायडू
एम. मुकुंदन
संदली सिन्हा
महाराणा प्रताप
लाल चिंतामणि शरण नाथ शाहदेव
एम॰ टी॰ वासुदेव नायर
फणि मुकुट राय
Saved batch 132 file
Working on batch 133
मेदिनी राय
Saved batch 262 file
Working on batch 263
ज्ञानपीठ पुरस्कार
शिल्पा शुक्ला
रामकरण मिश्र
Saved batch 241 file
Working on batch 242
के॰ जे॰ येशुदास
Saved batch 222 file
Working on batch 223
ईश्वर चन्द्र विद्यासागर
नीतू चन्द्रा
राम अवतार शर्मा
वी॰ नारायण मेनन
कावेरी झा
नमन ओझा
Saved batch 200 file
Working on batch 201
नलिन विलोचन शर्मा
गोविन्द शंकर कुरुप
नीलिमा अज़ीम
Saved batch 39 file
Working on batch 40
रावल मल्लिनाथ
श्रेया नारायण
बप्पा रावल
अल्लूरी सीताराम राजू
गंगानाथ झा
पूर्णिमा महतो
Saved batch 223 file
Working on batch 224
सृति झा
Saved batch 299 file
Working on batch 300
पुच्चलापल्ली सुंदरय्या
मृणालिनी साराभाई
गोपाल सिंह नेपाली
अल्का गुप्ता
ईशान किशन
रमेश चन्द्र झा
Saved batch 280 file
Working on batch 281
विकिमीडिया कॉमन्स
ओय्यारत्तु चंतु मेनोन
कोन्नेगंती हनुमंतु
चन्द्रशेखर आज़ाद
छवि पांडे
ओ. वी. विजयन
Saved batc

सी एन करुणाकरन
एम॰ एस॰ स्वामीनाथन
कविता बालकृष्णन
महाराणा कुम्भा
के शंकर पिल्लई
अक्किनेनी नागेश्वर राव
स्टैनफोर्ड विश्वविद्यालय
भारत
Saved batch 273 file
Working on batch 274
अनिल्डा थॉमस
डी रामानायडू
अरुणोदय सिंह
अंजू बॉबी जॉर्ज
कृष्णा (अभिनेता)
Saved batch 328 file
Working on batch 329
आशुतोष राणा
वी० ओ० चिदम्बरम पिल्लै
Saved batch 314 file
Working on batch 315
नेरेला वेणुमाधव
कसीनथुनी विश्वनाथ
पुल्लेला गोपीचंद
Saved batch 289 file
Working on batch 290
लता मंगेशकर
महाराजा जसवंत सिंह (मारवाड़)
बॉबी एलॉयसियस
सुब्रह्मण्य भारती
कर्णम मल्लेश्वरी
तिरुपुर कुमारन
जिसना मैथ्यू
Saved batch 43 file
Working on batch 44
Saved batch 315 file
Working on batch 316
गंगा सिंह
हम्पी कोनेरु
Saved batch 274 file
Working on batch 275
मोहम्मद अनास
अचंत शरत कमल
ओ॰ पी॰ जैशा
के. कामराज
प्रीजा श्रीधरन
जया बच्चन
Saved batch 329 file
Working on batch 330
वेलुपिल्लई प्रभाकरण
शाइनी अब्राहम
सूरज मल
भरत रेड्डी
Saved batch 275 file
Working on batch 391
बिधु भूषण दास
वरदराजन मुदलियार
केशवचंद्र दाश
वेंकटपति राजू
Saved 

अमय खुरासिया
शीला कौल
गोपबंधु दास
Saved batch 298 file
Working on batch 483
कपिला वात्स्यायन
सितारा देवी
Saved batch 383 file
Working on batch 384
गोपीनाथ मोहंती
श्रीप्रकाश जायसवाल
गौरहरि दास
जगन्नाथ प्रसाद दास
ध्यानचंद सिंह
शिव प्रताप शुक्ल
Saved batch 461 file
Working on batch 462
श्याम चरण गुप्ता
सरस्वती सम्मान
राम प्रसाद नौटियाल
Saved batch 384 file
Working on batch 385
जयंत महापात्र
नवीन पटनायक
मानसी प्रधान
कृष्णास्वामी कस्तूरीरंगन
मनोज दास
श्‍याम लाल यादव
Saved batch 403 file
Working on batch 404
नीलमणि राउत्रे
पिनाकी मिश्र
मनोरंजन दास
सुरेश पासी
कुंवर दिग्विजय सिंह
Saved batch 385 file
Working on batch 386
प्रसन्न कुमार पटसानी
मोहम्मद अयूब
एम आर श्रीनिवासन
Saved batch 483 file
Working on batch 484
आचार्य चतुरसेन शास्त्री
पी॰ वी॰ नरसिम्हा राव
भुवनेश्वर कुमार
निर्मला देवी
Saved batch 339 file
Working on batch 340
चिदम्बरम् सुब्रह्मण्यम्
राजगोपाल चिदंबरम
Saved batch 404 file
Working on batch 405
Saved batch 355 file
Working on batch 356
रवि राय
द्रव नोदन प्रणाली केंद्र
राधानाथ रथ
ए

श्वेता तिवारी
विनोद कुमार शुक्ल
Saved batch 534 file
Working on batch 535
ताराशंकर बंद्योपाध्याय
विष्णु शर्मा
अशेष प्रसाद मित्र
Saved batch 467 file
Working on batch 468
अशोक अमृतराज
सिद्धार्थ निगम
पञ्चतन्त्र
Saved batch 361 file
Working on batch 362
मरियप्पन थान्गावेलु
रामप्रसाद सेन
सोनल चौहान
अर्जुन पुरस्कार
राजशेखर बोस
देवेन्द्रनाथ ठाकुर
विनीत कुमार सिंह
सुब्बारमन विजयलक्ष्मी
अमीय चक्रवर्ती
Saved batch 496 file
Working on batch 497
Saved batch 556 file
Working on batch 557
ज़ोया अफ़रोज़
शारदा देवी
श्रीलाल शुक्ल
एम॰जे॰ गोपालन
बंकिमचन्द्र चट्टोपाध्याय
Saved batch 362 file
Working on batch 363
रविचंद्रन अश्विन
विवेकी राय
अनुराग कश्यप
कन्नौज
अभयचरणारविंद भक्तिवेदांत स्वामी प्रभुपाद
इस्मत चुग़ताई
श्रीनिवासाराघवान वेंकटराघवन
कृष्णम्माचारी श्रीकांत
Saved batch 514 file
Working on batch 515
कानपुर नगर जिला
मुरली कार्तिक
राजा राममोहन राय
Saved batch 363 file
Working on batch 364
फरहान अख्तर
कोलकाता नाईट राइडर्स
Saved batch 535 file
Working on batch 536
शरत्चन्द्र चट्टोपाध्याय
Saved batch 446

बेग़म अख़्तर
मिनीषा लाम्बा
अरविंद केजरीवाल
बाघा यतीन
सिद्धार्थ मल्होत्रा
हरिप्रसाद चौरसिया
Saved batch 598 file
Working on batch 599
मृणाल सेन
कृति सैनॉन
चितरंजन दास
झुम्पा लाहिड़ी
लालमणि मिश्र
Saved batch 456 file
Working on batch 457
अमरमणि त्रिपाठी
Saved batch 478 file
Working on batch 479
आरिफ़ मोहम्मद ख़ान
दादासाहेब फाल्के पुरस्कार
विष्णु नारायण भातखंडे
साकिब सलीम
बेनी प्रसाद वर्मा
अमिताभ घोष
रासबिहारी बोस
अशोक तंवर
दीनदयाल उपाध्याय
हुमा क़ुरैशी
Saved batch 568 file
Working on batch 569
बिमल राय
Saved batch 541 file
Working on batch 542
अवतार सिंह भड़ाना
अमित चौधुरी
यो यो हनी सिंह
काकोरी
Saved batch 525 file
Working on batch 526
चित्रा बैनर्जी दिवाकरूणी
ऋषिकेश मुखर्जी
बंसी लाल
सुबोध घोष
सुकान्त भट्टाचार्य
Saved batch 542 file
Working on batch 543
विकास महाराज
Saved batch 599 file
Working on batch 600
बिमल कर
पुलकित सम्राट
बासु चटर्जी
शक्ति चट्टोपाध्याय
हिन्दुस्तान सोशलिस्ट रिपब्लिकन एसोसिएशन
कानपुर
चिदानन्द दासगुप्ता
Saved batch 569 file
Working on batch 570
अपर्णा सेन
Saved batch

Saved batch 590 file
Working on batch 591
मिहिर कुमार सेन
पृथीपाल सिंह मैनी
गुरु हर किशन
कुलदीप सिंह चांदपुरी
मौमा दास
मिलखा सिंह
पौलोमी घटक
गुरु तेग़ बहादुर
बाळासाहेब गंगाधर खेर
Saved batch 608 file
Working on batch 609
लेफ्टिनेंट जनरल
जगजीत सिंह अरोड़ा
कमल जीत संधू
Saved batch 676 file
Working on batch 677
करमबीर सिंह
अंकिता दास
होमी जहांगीर भाभा
Saved batch 652 file
Working on batch 653
बलजिन्दर सिंह
गुरु ग्रन्थ साहिब
Saved batch 591 file
Working on batch 592
सौम्यजीत घोष
नंद सिंह
Saved batch 609 file
Working on batch 610
लांस नायक
जीव मिलखा सिंह
करम सिंह
ज्योतिर्मोयी सिकदर
जिंदर महल
अर्जुन अटवाल
अवनीत कौर सिधु
अनिर्वण लाहिड़ी
Saved batch 653 file
Working on batch 654
अर्जुन पुरस्कार
निर्मलजीत सिंह सेखों
Saved batch 628 file
Working on batch 629
Saved batch 592 file
Working on batch 593
श्री चन्द्रमुनि
दीपा कर्मकार
सतनाम सिंह भमरा
गुरबचन सिंह सलारिया
माधवराव पेशवा
आशा देवी आर्ययानाकम
तीजनबाई
जगतसेठ
भालचंद्र दिगंबर गरवारे
भाई गुरदास
Saved batch 610 file
Working on batch 611
श्रद्धारा

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [22]:
mixed_corpus_list = []
n = len(names_parallel)

start = datetime.now()
for i in range(12):
    mixed_corpus_list.append(get_mixed_corpus(names_parallel['hindi'][i]))
end = datetime.now()

print(end-start)

नरेश चन्द्रा
राम नारायण
मनमोहन शर्मा
के॰ एल॰ श्रीमाली
नगेन्द्र सिंह
दौलत सिंह कोठारी
मोहन सिंह मेहता
माणिक्यलाल वर्मा
घनश्यामदास बिड़ला
पद्म भूषण
कोमल कोठारी
दुर्गा लाल
0:03:10.380694


4

In [23]:
names_parallel.columns

Index(['english', 'hindi'], dtype='object')

In [24]:
word_corpus_list

[[['इलाहाबाद विश्वविद्यालय', 'University of Allahabad'],
  ['पद्म विभूषण', 'Padma Vibhushan'],
  ['भारतीय', 'Indian'],
  ['भारतीय प्रशासनिक सेवा', 'Indian Administrative Service'],
  ['राजस्थान', 'Rajasthan']],
 [['आई॰ऍस॰बी॰ऍन॰', 'International Standard Book Number'],
  ['आकाशवाणी', 'All India Radio'],
  ['उदयपुर', 'Udaipur'],
  ['दिल्ली', 'Delhi'],
  ['पद्म विभूषण', 'Padma Vibhushan'],
  ['भारत', 'India'],
  ['भारतीय सिनेमा', 'Cinema of India'],
  ['महाराष्ट्र', 'Maharashtra'],
  ['मुम्बई', 'Mumbai'],
  ['लाहौर', 'Lahore'],
  ['सारंगी', 'Sarangi'],
  ['हिन्दुस्तानी शास्त्रीय संगीत', 'Hindustani classical music']],
 [['कैम्ब्रिज', 'Cambridge'],
  ['जोधपुर', 'Jodhpur'],
  ['पद्म भूषण', 'Padma Bhushan'],
  ['भारत', 'India'],
  ['भारत सरकार', 'Government of India'],
  ['महाराष्ट्र', 'Maharashtra'],
  ['राजस्थान', 'Rajasthan'],
  ['१९८६', '1986']],
 [['उदयपुर', 'Udaipur'],
  ['कलकत्ता विश्वविद्यालय', 'University of Calcutta'],
  ['न्यूयॉर्क', 'New York (state)'],
  ['पद्म विभूषण', 'Padma V